In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf

In [6]:
foot_df = pd.read_pickle('/home/s232713/data/trajectories/FINAL_foot_data.pkl')
env_df = pd.read_csv('/home/s232713/data/foot_time_env.csv')

# print(foot_df.columns.values)
# print(20*'-')
# print(env_df.columns.values)

In [9]:
# -------------------------
# 1) Sanity check: env granularity
# -------------------------
print("env rows per (INDIVID, Interval ID) summary:")
print(env_df.groupby(["INDIVID", "Interval ID"]).size().describe())
print()

# -------------------------
# 2) Aggregate env_df -> one row per interval (time-weighted)
# -------------------------
env = env_df.copy()

env["t_entry"] = pd.to_datetime(env["t_entry"], errors="coerce")
env["t_exit"]  = pd.to_datetime(env["t_exit"], errors="coerce")

env["dur_s"] = (env["t_exit"] - env["t_entry"]).dt.total_seconds()

# fallback if numeric
mask_bad = env["dur_s"].isna()
if mask_bad.any():
    te = pd.to_numeric(env.loc[mask_bad, "t_entry"], errors="coerce")
    tx = pd.to_numeric(env.loc[mask_bad, "t_exit"], errors="coerce")
    env.loc[mask_bad, "dur_s"] = tx - te

env = env.replace([np.inf, -np.inf], np.nan)
env = env[env["dur_s"].notna() & (env["dur_s"] > 0)].copy()

ndvi_mean = "grid_ndvi_mean"
ndvi_max  = "grid_ndvi_max"
mapillary = ["Vegetation", "Sky", "Curb"]

pois = [
    "tourism_leisure",
    "arts_entertainment_events",
    "greenery_natural",
    "water_body",
    "roads_transportation",
    "buildings_facilities",
    "public_services",
    "food_drink",
    "miscellaneous_services",
    "beauty_personal_health",
]
controls = ["Slope"]

pred_cols = [ndvi_mean, ndvi_max] + mapillary + pois + controls
pred_cols = [c for c in pred_cols if c in env.columns]

# Fill NA with 0 for share/count-like predictors (NOT for *_max)
fill0_cols = [c for c in pred_cols if c != ndvi_max]
env[fill0_cols] = env[fill0_cols].fillna(0)

def wmean(x, w):
    if x is None:
        return np.nan
    ok = x.notna() & w.notna() & (w > 0)
    if not ok.any():
        return np.nan
    return np.average(x[ok], weights=w[ok])

g = ["INDIVID", "Interval ID"]

def agg_group(d):
    out = {}

    if ndvi_mean in d.columns:
        out["NDVI_mean"] = wmean(d[ndvi_mean], d["dur_s"])
    if ndvi_max in d.columns:
        out["NDVI_max"] = d[ndvi_max].max()

    for c in mapillary:
        if c in d.columns:
            out[f"Mapillary_{c}"] = wmean(d[c], d["dur_s"])

    for c in pois:
        if c in d.columns:
            out[c] = wmean(d[c], d["dur_s"])

    for c in controls:
        if c in d.columns:
            out[c] = wmean(d[c], d["dur_s"])

    out["interval_dur_s"] = d["dur_s"].sum()
    return pd.Series(out)

# note: the FutureWarning is harmless, but this avoids it on new pandas
try:
    env_trip = env.groupby(g, group_keys=False).apply(agg_group).reset_index()
except TypeError:
    env_trip = env.groupby(g).apply(agg_group).reset_index()

print("env_trip shape:", env_trip.shape)
print("env_trip columns:", env_trip.columns.tolist())
print()

# -------------------------
# 3) Merge outcomes from diary
# -------------------------
outcome_cols = [
    "tripFeelingsEnvironment",
    "tripFeeling_Well",
    "tripFeeling_Calm",
    "tripFeeling_Awake",
]

need_diary = ["INDIVID", "Interval ID"] + [c for c in outcome_cols if c in foot_df.columns]
diary = foot_df[need_diary].copy()

df_h1 = diary.merge(env_trip, on=["INDIVID", "Interval ID"], how="inner")

# --- FIX: force outcomes numeric (prevents dummy-expanded endog) ---
for c in outcome_cols:
    if c in df_h1.columns:
        df_h1[c] = pd.to_numeric(df_h1[c], errors="coerce")

df_h1 = df_h1.dropna(subset=[c for c in outcome_cols if c in df_h1.columns]).copy()

print("df_h1 shape after merge+dropna outcomes:", df_h1.shape)
print("Outcome dtypes:\n", df_h1[outcome_cols].dtypes)
print()

# -------------------------
# 4) Standardize predictors (z-score)
# -------------------------
predictors = []
for c in ["NDVI_mean", "NDVI_max", "Mapillary_Vegetation", "Mapillary_Sky", "Mapillary_Curb"]:
    if c in df_h1.columns:
        predictors.append(c)

for c in pois + controls:
    if c in df_h1.columns and c not in predictors:
        predictors.append(c)

# drop all-NaN or constant predictors
clean_preds = []
for c in predictors:
    s = df_h1[c]
    if s.notna().sum() == 0:
        continue
    if s.nunique(dropna=True) <= 1:
        continue
    clean_preds.append(c)
predictors = clean_preds

scaler = StandardScaler()
Z = scaler.fit_transform(df_h1[predictors].astype(float))
z_cols = [f"z_{c}" for c in predictors]
df_h1[z_cols] = Z

print("Predictors used:", predictors)
print()

# -------------------------
# 5) Fit MixedLM (Lisbon-style)
# -------------------------
import statsmodels.api as sm

def fit_mixed_with_fallback(formula, df, group_col="INDIVID"):
    # 1) Try MixedLM with a few optimizers
    for method in ["lbfgs", "powell", "nm"]:
        try:
            m = smf.mixedlm(formula, df, groups=df[group_col])
            r = m.fit(reml=False, method=method, disp=False)
            # if it converged but still boundary-ish, sometimes summary can still break later,
            # but usually this passes
            return ("MixedLM", method, r)
        except np.linalg.LinAlgError:
            continue
        except ValueError:
            continue

    # 2) Fallback: OLS with clustered SE by participant
    r = smf.ols(formula, df).fit(cov_type="cluster", cov_kwds={"groups": df[group_col]})
    return ("OLS_cluster", "cluster", r)


rows = []

for label, y in outcomes.items():
    if y not in df_h1.columns:
        print(f"Skipping outcome {label} because column {y} not found")
        continue

    df_y = df_h1.dropna(subset=[y]).copy()
    if df_y[y].nunique() <= 1:
        print(f"Skipping outcome {label} because {y} is constant after cleaning")
        continue

    # optional but helps MixedLM: drop groups with <2 obs
    df_y = df_y.groupby("INDIVID").filter(lambda d: len(d) >= 2).copy()
    if df_y.empty:
        print(f"Skipping outcome {label} because no groups with >=2 obs")
        continue

    formula = f"{y} ~ " + " + ".join(z_cols)

    model_type, method, res = fit_mixed_with_fallback(formula, df_y, group_col="INDIVID")

    print(f"\n=== {model_type} outcome: {label} ({y}) | method={method} ===")
    try:
        print(res.summary())
    except Exception:
        print(res.params)

    # extract coefficients for the standardized predictors only
    for zp in z_cols:
        rows.append({
            "outcome": label,
            "model_type": model_type,
            "optimizer": method,
            "predictor": zp.replace("z_", ""),
            "beta": float(res.params.get(zp, np.nan)),
            "se": float(res.bse.get(zp, np.nan)) if hasattr(res, "bse") else np.nan,
            "pval": float(res.pvalues.get(zp, np.nan)) if hasattr(res, "pvalues") else np.nan,
            "n_obs": int(res.nobs) if hasattr(res, "nobs") else int(df_y.shape[0]),
            "n_individ": int(df_y["INDIVID"].nunique()),
        })

h1_results = pd.DataFrame(rows).sort_values(["outcome", "pval"])
out_path = "CPH_H1_mixedlm_results.csv"
h1_results.to_csv(out_path, index=False)

print("\nSaved:", out_path)
print(h1_results.head(30))

env rows per (INDIVID, Interval ID) summary:
count    1973.000000
mean       17.107451
std        23.194644
min         1.000000
25%         7.000000
50%        11.000000
75%        18.000000
max       578.000000
dtype: float64



/tmp/ipykernel_4181458/2690137869.py:88: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  env_trip = env.groupby(g, group_keys=False).apply(agg_group).reset_index()


env_trip shape: (1962, 18)
env_trip columns: ['INDIVID', 'Interval ID', 'NDVI_mean', 'NDVI_max', 'Mapillary_Vegetation', 'Mapillary_Sky', 'Mapillary_Curb', 'tourism_leisure', 'arts_entertainment_events', 'greenery_natural', 'water_body', 'roads_transportation', 'buildings_facilities', 'public_services', 'food_drink', 'miscellaneous_services', 'beauty_personal_health', 'interval_dur_s']

df_h1 shape after merge+dropna outcomes: (687, 22)
Outcome dtypes:
 tripFeelingsEnvironment    float64
tripFeeling_Well           float64
tripFeeling_Calm           float64
tripFeeling_Awake          float64
dtype: object

Predictors used: ['NDVI_mean', 'NDVI_max', 'Mapillary_Vegetation', 'Mapillary_Sky', 'Mapillary_Curb', 'tourism_leisure', 'arts_entertainment_events', 'greenery_natural', 'water_body', 'roads_transportation', 'buildings_facilities', 'public_services', 'food_drink', 'miscellaneous_services', 'beauty_personal_health']


=== MixedLM outcome: tripFeelingsEnvironment (tripFeelingsEnvironmen

/home/s232713/.local/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


## Copenhagen (CPH) – H1 MixedLM: what we can say (without Lisbon comparison)

**Model / setup**
- Replicates a “Lisbon-style” mixed-effects regression at the *interval* level.
- Outcomes (diary, interval-level):  
  - `tripFeelingsEnvironment`, `tripFeeling_Well`, `tripFeeling_Calm`, `tripFeeling_Awake`
- Predictors (interval-level exposures):  
  - NDVI: `NDVI_mean` (time-weighted), `NDVI_max` (max within interval)  
  - Mapillary shares (time-weighted): `Mapillary_Vegetation`, `Mapillary_Sky`, `Mapillary_Curb`  
  - POI category intensities (time-weighted): tourism/leisure, arts/events, greenery/natural, water, roads/transport, buildings/facilities, public services, food/drink, misc services, beauty/health  
  - Control: `Slope`
- Each predictor is z-scored across the analysis dataset (so coefficients are “per 1 SD increase”).
- Random intercept by participant: `groups = INDIVID` (accounts for baseline differences between people).

**Data handling choices (important for interpretation)**
- Exposures are aggregated to one row per `(INDIVID, Interval ID)` using time spent in each cell segment (`dur_s`).
- Many exposure-like variables are filled with 0 when missing (interpreted as “no exposure / not present”), while `NDVI_max` is not filled.
- Intervals with invalid/zero duration are removed, and outcomes are coerced to numeric.

**How to interpret coefficients**
- A positive beta means: higher exposure during the interval is associated with higher outcome rating (on its original scale), *within the mixed model*.
- Because predictors are standardized, magnitudes are comparable across predictors: larger |beta| = stronger association per SD.

**What to report from the CPH run**
- Report `n_obs` (intervals kept) and `n_individ` (participants contributing) for each outcome.
- Highlight the predictors with the smallest p-values per outcome (e.g., top 3–5), and note the direction (positive/negative).
- Avoid overselling: this is observational, interval-aggregated, and the model includes many correlated predictors, so single-coefficient interpretation can be unstable.

**Common patterns to check before writing conclusions**
- Do the same predictors appear across outcomes (Environment vs Well/Calm/Awake), or is it outcome-specific?
- Are signs consistent with intuition? (e.g., more `roads_transportation` -> worse environment, more `Mapillary_Vegetation`/NDVI -> better environment/calm)
- Are effects tiny even when “significant”? If yes, emphasize practical insignificance.
- If many predictors look significant, consider multiple-testing / collinearity as an explanation, not “lots of real effects”.

**Suggested wording if results are mostly weak / inconsistent**
- “Under this Lisbon-style interval-level MixedLM, we find limited evidence of stable associations between environmental exposures and diary outcomes in Copenhagen. Effects that appear statistically detectable are generally small, and their direction and presence vary across outcomes, suggesting substantial noise and/or participant- and context-specific responses at this temporal support.”
